In [ ]:
import os
import pandas as pd
import sys
import numpy as np
import sys
sys.path.insert(0, '../..')
import itertools

from JKBio import TerraFunction as terra
from ccle_processing.src.CCLE_postp_function import *
from JKBio import Helper as h
from JKBio.epigenetics import ChIP_helper as chip

import dalmatian as dm
import pyBigWig

from bokeh.plotting import *
from IPython.display import IFrame
import igv

from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture

output_notebook()
%load_ext autoreload
%autoreload 2

In [ ]:
rename1 = {
'gs://amlproject/slamseq/IRF2BP2/20191212_10_MP7623_S30_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp810_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_10_MP7623_S30_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp810_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_11_MP7623_S31_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp811_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_11_MP7623_S31_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp811_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_12_MP7623_S32_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp812_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_12_MP7623_S32_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp812_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_13_MP7623_S33_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp813_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_13_MP7623_S33_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp813_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_14_MP7623_S34_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp814_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_14_MP7623_S34_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp814_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_15_MP7623_S35_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp815_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_15_MP7623_S35_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp815_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_16_MP7623_S36_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp816_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_16_MP7623_S36_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp816_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_1_MP7623_S21_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp801_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_1_MP7623_S21_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp801_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_2_MP7623_S22_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp802_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_2_MP7623_S22_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp802_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_3_MP7623_S23_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp803_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_3_MP7623_S23_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp803_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_4_MP7623_S24_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp804_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_4_MP7623_S24_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp804_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_5_MP7623_S25_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp805_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_5_MP7623_S25_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp805_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_6_MP7623_S26_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp806_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_6_MP7623_S26_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp806_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_7_MP7623_S27_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp807_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_7_MP7623_S27_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp807_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_8_MP7623_S28_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp808_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_8_MP7623_S28_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp808_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_9_MP7623_S29_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp809_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_9_MP7623_S29_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp809_2.fastq.gz'}

In [ ]:
for k, val in rename1.items():
    os.system('gsutil mv '+k+' '+val)

In [ ]:
terra.list_blobs_with_prefix("amlproject","slamseq/IRF2BP2/")

In [ ]:
terra.uploadFromFolder("amlproject", "slamseq/IRF2BP2/", workspace="broad-firecloud-ccle/hg38_RNAseq", 
                       fformat="fastq12", source="maximP",samplesetname="slamseq_IRF2BP2")

In [ ]:
samplesetname = 'slamseq_IRF2BP2'
refworkspace="broad-firecloud-ccle/hg38_RNAseq"
refwm = dm.WorkspaceManager(refworkspace)

In [ ]:
submission_id = refwm.create_submission("slamdunk", samplesetname,'sample_set',expression='this.samples')
terra.waitForSubmission(refworkspace, submission_id)

In [ ]:
submission_id1 = refwm.create_submission("rsem_v1-0_BETA_cfg", samplesetname,'sample_set',expression='this.samples')
terra.waitForSubmission(refworkspace, submission_id1)

In [ ]:
! pip3 install git+https://github.com/jkobject/slamdunk.git --upgrade

In [ ]:
folder_bams = "../../data/slam/data/"
bams = ! ls $folder_bams*.fastq.gz
bams

In [ ]:
for val in h.grouped(bams[4:],2):
    print(val)
    res = os.system('../../TrimGalore-0.6.5/trim_galore --paired --cores 12 --retain_unpaired -stringency 3 --illumina '+
    val[0] + ' '+val[1]+" -o "+folder_bams)
    if res!=0:
        raise ValueError()
    res = os.system('slamdunk all -r ../../data/slam/ref_genome.fa -b ../../data/slam/GSE100708_hg38_refseq_062016_ensemblv84_3UTR.bed \
    -o ../../data/slam/resIRF2BP2/ -t 12 -c 2  '+'.'.join(val[0].split('.')[:-2])+'_val_1.fq.gz \
    '+'.'.join(val[1].split('.')[:-2])+"_val_2.fq.gz")
    if res!=0:
        raise ValueError()

In [ ]:
for val in h.grouped(bams[:4],2):
    print(val)
    res = os.system('slamdunk all -r ../../data/slam/ref_genome.fa -b ../../data/slam/GSE100708_hg38_refseq_062016_ensemblv84_3UTR.bed \
    -o ../../data/slam/resIRF2BP2/ -t 12 -c 2  '+'.'.join(val[0].split('.')[:-2])+'_val_1.fq.gz \
    '+'.'.join(val[1].split('.')[:-2])+"_val_2.fq.gz")
    if res!=0:
        raise ValueError()